### Quality Check using GPT-4o-mini

In [1]:
import os
from openai import OpenAI
from utils.utils import write_jsonl, read_problems


def make_gpt_chat_messsage(role, content):
    return {'role': role, 'content': content}

def make_prompt(a, b):
    return f'''\
### Instruction
Given a docstring of Python function written in English and its Korean translation, please rate how well the translation conveys the original meaning using an integer between 1 and 3 (with 3 being the best). No explanation is necessary; just provide a rating between 1 and 3.

### English docstring
{a}

### translated Korean docstring
{b}

### Rating
'''


he = read_problems('HumanEval')
he_kr = read_problems('KR_HumanEval')

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

save_path = os.path.join(None) # assign!
        
for k1, k2 in zip(he, he_kr):
    original_prompt = he[k1]['prompt']
    translated_prompt = he_kr[k2]['prompt']
    
    messages = []
    messages.append(make_gpt_chat_messsage('system', 'You are a helpful AI assistant.'))
    messages.append(make_gpt_chat_messsage('user', make_prompt(original_prompt, translated_prompt)))
    
    completion = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        n=1,
        max_tokens=32,
    )
    response = completion.choices[0].message.content
    
    # print('original prompt:')
    # print(original_prompt)
    # print('translated prompt:')
    # print(translated_prompt)
    # print(f'rate: {response}')
    # print('-' * 100)
    
    # add a score
    he_kr[k2]['translation_score'] = response
    result = []
    result.append(he_kr[k2])

    write_jsonl(save_path, result, append=True)